In [1]:
import xarray as xr
import rioxarray as rioxr
import geopandas as gpd
import fiona
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import os
import glob
import datetime
import tqdm
import tqdm.notebook
from shapely import wkt
import dask
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt

In [2]:
cluster = LocalCluster(n_workers=8, memory_limit="4gb")

In [3]:
cluster

In [4]:
client = Client(cluster)

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52068,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:52105,Total threads: 2
Dashboard: http://127.0.0.1:52109/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:52071,


In [6]:
SCHEMA = pa.schema([('time', pa.timestamp('s')),
                    ('latitude', pa.float64()),
                    ('longitude', pa.float64()),
                    ('adm_0', pa.string()),
                    ('adm_1', pa.string()),
                    ('adm_2', pa.string()),
                    ('t2m_min', pa.float64()),
                    ('t2m_max', pa.float64()),
                    ('t2m', pa.float64()),
                    ('ssr', pa.float64()),
                    ('ssr_min', pa.float64()),
                    ('ssr_max', pa.float64()),
                    ('tp_min', pa.float64()),
                    ('tp_max', pa.float64()),
                    ('tp', pa.float64()),
                   ])

DF_COLS = [
    'time','longitude','latitude','adm_0','adm_1','adm_2','t2m_min','t2m_max','t2m','ssr_min','ssr_max','ssr','tp_min','tp_max','tp'
]

In [7]:
df = gpd.read_file("gadm41_BRA.gpkg", layer="ADM_ADM_2")

2023-05-28 20:40:34,035 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-05-28 20:40:34,252 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2023-05-28 20:40:34,528 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2023-05-28 20:40:34,733 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2023-05-28 20:40:34,946 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2023-05-28 20:40:35,216 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2023-05-28 20:40:35,427 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2023-05-28 20:40:35,617 - distributed.utils_perf - WARNING - full garbage collections took

2023-05-28 20:40:57,940 - distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
2023-05-28 20:40:58,257 - distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
2023-05-28 20:40:58,631 - distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
2023-05-28 20:40:59,182 - distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)


In [8]:
df = df.query("NAME_1 == 'Mato Grosso'")

In [9]:
#layers = fiona.listlayers('gadm41_BRA.gpkg')
#layers = ['ADM_ADM_0', 'ADM_ADM_1', 'ADM_ADM_2']

In [10]:
years = [str(x) for x in range(2005, 2024)]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
agg_cols = {'t2m': np.mean, 'ssr': np.sum, 'tp': np.sum}
filename_pattern = './output/cut_mt/{year}_{month}.parquet'
ds = None

#years = ['2005']
#months = ['01']

In [14]:
for year in years:
    for month in months:
        print(year, month)
        
        filename = filename_pattern.format(year=year, month=month)
        
        if os.path.exists(filename):
            print(filename)
            continue
        
        parquet_writer = pq.ParquetWriter(filename, SCHEMA, version="1.0")
        if 1 ==1:
            ds = xr.open_mfdataset('./output/era5/era5_{year}_{month}.nc'.format(year=year, month=month),
                                   chunks={'longitude': 50, 'latitude': 50, 'time': 100},
                                   #engine='h5netcdf',
                                   #parallel=True
                                  )
            ds = ds.rio.write_crs('EPSG:4326', inplace=True)
            #print("inicio - ds.load()")
            #ds = ds.load()
            #print("fim - ds.load()")

        for idx in tqdm.notebook.tqdm(df.index):
            geom = df.loc[idx]

            try:
                ds_cut = ds.rio.clip([geom.geometry],
                                    'EPSG:4326',
                                    all_toutched=True).mean(dim=('longitude', 'latitude')).to_dataframe().reset_index().dropna()
            except:
                try:
                    centroid_aux = geom.geometry.centroid
                    aux = ds.sel(latitude=centroid_aux.y,
                                 longitude=centroid_aux.x,
                                 method='nearest')
                    ds_cut = aux.to_dataframe().reset_index().dropna()
                except:
                    print('nao deu nao...', geom)
            
            _ = ds_cut.pop("spatial_ref")
            
            ds_cut['adm_0'] = geom.COUNTRY
            ds_cut['adm_1'] = geom.NAME_1
            ds_cut['adm_2'] = geom.NAME_2
            
            #converte Kelvin para Celcius
            ds_cut['t2m'] = ds_cut['t2m'] - 273.15
            
            ds_cut['latitude'] = geom.geometry.centroid.y
            ds_cut['longitude'] = geom.geometry.centroid.x

            ds_cut['time'] = pd.to_datetime(ds_cut['time'])
            
            # gravando agregado por dia para salvar espaco em disco
            ds_cut['time'] = ds_cut['time'].dt.date
            ds_cut['time'] = pd.to_datetime(ds_cut['time'])
            
            
            ds_cut = ds_cut.groupby(['time', 'longitude', 'latitude', 'adm_0', 'adm_1', 'adm_2']).agg(
                t2m_min=pd.NamedAgg(column="t2m", aggfunc=np.min),
                t2m_max=pd.NamedAgg(column="t2m", aggfunc=np.max),
                t2m=pd.NamedAgg(column="t2m", aggfunc=np.mean),
                ssr_min=pd.NamedAgg(column="ssr", aggfunc=np.min),
                ssr_max=pd.NamedAgg(column="ssr", aggfunc=np.max),
                ssr=pd.NamedAgg(column="ssr", aggfunc=np.sum),
                tp_min=pd.NamedAgg(column="tp", aggfunc=np.min),
                tp_max=pd.NamedAgg(column="tp", aggfunc=np.max),
                tp=pd.NamedAgg(column="tp", aggfunc=np.sum),
            ).reset_index()
            
            parquet_writer.write_table(pa.Table.from_pandas(ds_cut[DF_COLS], schema=SCHEMA))
        parquet_writer.close()


2005 01


  0%|          | 0/141 [00:00<?, ?it/s]

2005 02


  0%|          | 0/141 [00:00<?, ?it/s]

2005 03


...

  0%|          | 0/141 [00:00<?, ?it/s]

2023 05


  0%|          | 0/141 [00:00<?, ?it/s]

2023 06


OSError: no files to open

In [15]:
parquet_writer.close()